# Loading necessary libraries

In [ ]:
from PIL import Image
import numpy as np
from random import randrange
import random
import h5py
import scipy.io
import os
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import torch
import pickle
import imgaug.augmenters as iaa 
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms.functional import InterpolationMode
import torch.optim as optim

# Mounting Drive to the Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Preparing Train data & Test data

## Train data

In [ ]:
root_img = r"/content/drive/My Drive/images"
root_label = r"/content/drive/My Drive/ground-truth"

In [ ]:
images_folder = root_img
images_names = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]

In [ ]:
random.seed(10)
matrix = 260
sample = 9
sample_list = []
label_dict = []
k = 0

for image_name in images_names:
    img = Image.open(root_img + "/" + image_name)
    mat = scipy.io.loadmat(root_label + "/" + 'GT_' + image_name[:-4])
    coordinates = mat['image_info'][0,0][0,0][0]
    x, y = img.size
    for i in range(sample):
        x1 = randrange(0, x - matrix)
        y1 = randrange(0, y - matrix)
        sample_list.append(img.crop((x1, y1, x1 + matrix, y1 + matrix)))
        label_list = []
        for j in range(coordinates.shape[0]):
            x_label = coordinates[j,0]
            y_label = coordinates[j,1]
            if x_label >= x1 and x_label <= x1+matrix and y_label >= y1 and y_label <= y1 + matrix:
                x_new = (x_label - x1)/4
                y_new = (y_label - y1)/4
                label_list.append([int(x_new), int(y_new)])
        label_dict.append(label_list)           
        k += 1            
        

In [ ]:
random.seed(10)

dens_list = []
for i in range (len(label_dict)):
    dens_map = torch.zeros((65,65))
    for j in label_dict[i]:
        dens_map[j[1], j[0]] = 1
    dens_list.append(torch.from_numpy(gaussian_filter(dens_map, sigma = 3, mode = 'constant')))    

### Saving Train data in Drive

In [ ]:
# save data
with open("dens_train.txt", "wb") as fp:   #Pickling
    pickle.dump(dens_list, fp)    


In [ ]:
!cp "/content/dens_train.txt" "/content/drive/My Drive"

In [ ]:
# save data
with open("img_train.txt", "wb") as fp:   #Pickling
    pickle.dump(sample_list, fp)   

In [ ]:
!cp "/content/img_train.txt" "/content/drive/My Drive"

### Showing a sample of Train data

In [ ]:
plt.imshow(sample_list[2])
plt.show()
plt.imshow(dens_list[2])
plt.show()

## Test data

In [ ]:
root_img = r"/content/drive/My Drive/images-test"
root_label = r"/content/drive/My Drive/ground-truth-test"

In [ ]:
images_folder = root_img
images_names = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]

In [ ]:
random.seed(10)
matrix = 260
sample = 9
sample_list_test = []
label_dict_test = []
k = 0

for image_name in images_names:
    img = Image.open(root_img + "/" + image_name)
    mat = scipy.io.loadmat(root_label + "/" + 'GT_' + image_name[:-4])
    coordinates = mat['image_info'][0,0][0,0][0]
    x, y = img.size
    for i in range(sample):
        x1 = randrange(0, x - matrix)
        y1 = randrange(0, y - matrix)
        sample_list_test.append(img.crop((x1, y1, x1 + matrix, y1 + matrix)))
        label_list = []
        for j in range(coordinates.shape[0]):
            x_label = coordinates[j,0]
            y_label = coordinates[j,1]
            if x_label >= x1 and x_label <= x1+matrix and y_label >= y1 and y_label <= y1 + matrix:
                x_new = (x_label - x1)/4
                y_new = (y_label - y1)/4
                label_list.append([int(x_new), int(y_new)])
        label_dict_test.append(label_list)           
        k += 1            

In [ ]:
random.seed(10)

dens_list_test = []
for i in range (len(label_dict_test)):
    dens_map = torch.zeros((65,65))
    for j in label_dict_test[i]:
        dens_map[j[1], j[0]] = 1
    dens_list_test.append(torch.from_numpy(gaussian_filter(dens_map, sigma = 3, mode = 'constant'))) 

### Saving Test data in Drive

In [ ]:
# save data
with open("dens_test.txt", "wb") as fp:   #Pickling
    pickle.dump(dens_list_test, fp) 

!cp "/content/dens_test.txt" "/content/drive/My Drive"       

In [ ]:
# save data
with open("img_test.txt", "wb") as fp:   #Pickling
    pickle.dump(sample_list_test, fp)   

!cp "/content/img_test.txt" "/content/drive/My Drive"    

# Loading Train data & Test data from Drive

In [ ]:
# load data

with open(r"/content/drive/My Drive/dens_train.txt", "rb") as fp:
  dens_list = pickle.load(fp)

with open(r"/content/drive/My Drive/img_train.txt", "rb") as fp:
  sample_list = pickle.load(fp)  

In [ ]:
# load data

with open(r"/content/drive/My Drive/dens_test.txt", "rb") as fp:
  dens_list_test = pickle.load(fp)

with open(r"/content/drive/My Drive/img_test.txt", "rb") as fp:
  sample_list_test = pickle.load(fp)  

In [ ]:
sample_list += sample_list_test
dens_list += dens_list_test

# Dataloader class

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, list_IDs,img_transform, aug=False):
        self.list_IDs = list_IDs
        self.img_transform = img_transform
        self.aug = aug

  def __len__(self):
        return len(self.list_IDs)

  def __getitem__(self, ind):

        # Load data and get label
        X = self.load_img(ind)
        y = self.load_label(ind)

        return X, y

  def load_img(self, ind):
    if self.aug == False:
        img = sample_list[ind]
    else: 
        img = new_images[ind]

    transformed_img = self.img_transform(img) 
    return transformed_img

  def load_label(self, ind):
    if self.aug == False:
        dens_map = dens_list[ind]
    else:
        dens_map = new_dens[ind]

    return dens_map

# Convolutional Neural Network class

In [ ]:
class CCNN(nn.Module):
    def __init__(self):
        super(CCNN, self).__init__()
      # convolutional layers
        self.conv1 = nn.Conv2d(3, 32, 11, padding = 5)
        self.conv2 = nn.Conv2d(32, 32, 7, padding = 3)
        self.conv3 = nn.Conv2d(32, 64, 5, padding = 2)
        self.conv4 = nn.Conv2d(64, 1000, 1, padding = 0)
        self.conv5 = nn.Conv2d(1000, 400, 1, padding = 0)
        self.conv6 = nn.Conv2d(400, 1, 1, padding = 0)
        
      # max pooling layers in encoder
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 2)
      

    def forward(self,x):
       # First layer 
        x = F.relu(self.conv1(x))

       # Second layer
        x = self.pool1(F.relu(self.conv2(x)))
 
       # Third layer
        x = self.pool2(F.relu(self.conv3(x)))

       # Fourth layer
        x = F.relu(self.conv4(x))

       # Fifth layer
        x = F.relu(self.conv5(x))

       # Sixth layer
        x = F.relu(self.conv6(x))

    
        return x

# Part 1: Training without data augmention

## Defining Train loader, Validation loader, and Test loader

In [ ]:
random.seed(10)

train_indices = [] 
valid_indices = random.sample(range(0, 3600), 360)
for i in range(len(dens_list)):
    if i not in valid_indices:
        train_indices.append(i)

test_indices = [len(dens_list), len(dens_list)+len(dens_list_test)]        

partition = {'train' : train_indices , 'validation' : valid_indices, 'test' : test_indices}

In [ ]:
training_set = Dataset(partition['train'], transform)

training_loader = torch.utils.data.DataLoader(training_set, batch_size=30,
                                             shuffle=True, num_workers=2)

validation_set = Dataset(partition['validation'], transform)

validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=30,
                                               shuffle=True, num_workers=2)

## Training the network 

### Specifying training options

In [ ]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [ ]:
torch.manual_seed(10)
# Model 
Net = CCNN()
Net.to(device)

In [ ]:
torch.manual_seed(10)

def lossfunc(output, target):
    return (torch.dist(output, target,p=2) /(2*output.shape[0]))**2

In [ ]:
torch.manual_seed(10)

optimizer = optim.Adam(Net.parameters(), lr = 0.00005)

In [ ]:
max_epochs = 10

In [ ]:
transform = transforms.ToTensor()

In [ ]:
mse = nn.MSELoss()
mae = nn.L1Loss()

### Training main loop

In [ ]:
torch.manual_seed(10)

train_losses = []
validation_losses = []
mse_train_losses = []
mse_valid_losses = []
mae_train_losses = []
mae_valid_losses = []

# Loop over epochs
for epoch in range(max_epochs):
    train_loss = 0
    valid_loss = 0
    batch_loss_train = 0
    batch_loss_valid = 0

    train_mse = 0
    valid_mse = 0
    batch_mse_train = 0
    batch_mse_valid = 0

    train_mae = 0
    valid_mae = 0
    batch_mae_train = 0
    batch_mae_valid = 0

    train_counter = 0
    val_counter = 0 
    
    # Training
    for batch_data, batch_labels in training_loader:

        # Transfer to GPU
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # Model computations
        batch_outputs = Net(batch_data)        
        loss = lossfunc(batch_outputs, batch_labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        label = torch.round(torch.sum(batch_labels, dim = [1,2]))
        output = torch.round(torch.sum(batch_outputs.squeeze(), dim = [1,2]))

        train_mse += torch.sqrt(mse(label,output))
        train_mae += torch.sqrt(mae(label,output))

        # print statistics
        

        print('Training: ', 'Epoch No: ', epoch+1, 'Iteration No: ', train_counter+1, '\n',
              'Loss: ', train_loss)
        batch_loss_train += train_loss
        batch_mse_train += train_mse
        batch_mae_train += train_mae  
        train_loss = 0
        train_mse = 0
        train_mae = 0      
        train_counter += 1
        
    # Validation
    with torch.set_grad_enabled(False):
        for batch_data, batch_labels in validation_loader:
            # Transfer to GPU
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
            
            # Model computations
            batch_outputs = Net(batch_data)            
            loss = lossfunc(batch_outputs, batch_labels)
            valid_loss += loss.item()
             

            label = torch.round(torch.sum(batch_labels, dim = [1,2]))
            output = torch.round(torch.sum(batch_outputs.squeeze(), dim = [1,2]))

            valid_mse += torch.sqrt(mse(label,output))
            valid_mae += torch.sqrt(mae(label,output))

            print('Validation: ', 'Epoch No: ', epoch+1, 'Iteration No: ', val_counter+1, '\n',
                  'Loss: ', valid_loss)
            batch_loss_valid +=  valid_loss
            batch_mse_valid += valid_mse
            batch_mae_valid += valid_mae
            valid_loss = 0
            valid_mse = 0
            valid_mae = 0   
            val_counter += 1
    
    train_losses.append(batch_loss_train / train_counter)
    validation_losses.append(batch_loss_valid / val_counter)
    mse_train_losses.append(batch_mse_train / train_counter)
    mse_valid_losses.append(batch_mse_valid / val_counter)
    mae_train_losses.append(batch_mae_train / train_counter)
    mae_valid_losses.append(batch_mae_valid / val_counter)

### Saving trained network 


In [ ]:
PATH = '/content/drive/My Drive/CCNN.pth'
torch.save(Net.state_dict(), PATH)

### Showing Train & Validation Losses

In [ ]:
plt.plot(train_losses)
plt.plot(validation_losses)
plt.xlabel('Epoch No')
plt.ylabel('Averaged loss')
plt.title("CCNN loss function")
plt.legend(labels = ["train","validation"])
plt.show()

In [ ]:
plt.plot(mse_train_losses)
plt.plot(mse_valid_losses)
plt.xlabel('Epoch No')
plt.ylabel('Averaged MSE loss')
plt.title("CCNN MSE loss")
plt.legend(labels = ["train","validation"])
plt.show()

In [ ]:
plt.plot(mae_train_losses)
plt.plot(mae_valid_losses)
plt.xlabel('Epoch No')
plt.ylabel('Averaged MAE loss')
plt.title("CCNN MAE loss")
plt.legend(labels = ["train","validation"])
plt.show()

# Part 2: Training with data augmention

## Data Augmentation

In [ ]:
seq = iaa.Sequential([iaa.GammaContrast((0.5, 2.0)),
                                        iaa.Affine(scale={"x": (0.5, 1.5), "y": (0.5, 1)}),
                                        iaa.Affine(translate_px={"x": (-10, 10), "y": (-20, 10)}),
                                        iaa.Affine(rotate=(-45, 45))])

In [ ]:
aug_sample = []
aug_dens = []
l = len(sample_list)
aug_indices = random.sample(range(0, l), int(0.2*l))
for i in range(l):
  if i in aug_indices:
    image_aug = seq(images=np.array(sample_list[i]))
    dens_map_aug = seq(images = dens_list[i].numpy())
    aug_sample.append(Image.fromarray(image_aug))
    aug_dens.append(torch.from_numpy(dens_map_aug))


### Saving data in Drive

In [ ]:
# save data
with open("dens_train_aug.txt", "wb") as fp:   #Pickling
    pickle.dump(aug_dens, fp)    

with open("sample_train_aug.txt", "wb") as fp:   #Pickling
    pickle.dump(aug_sample, fp)        

In [ ]:
!cp "/content/dens_train_aug.txt" "/content/drive/My Drive"
!cp "/content/sample_train_aug.txt" "/content/drive/My Drive"

### Loading data from Drive

In [ ]:
# load data

with open(r"/content/drive/My Drive/dens_train_aug.txt", "rb") as fp:
  dens_aug_list = pickle.load(fp)

with open(r"/content/drive/My Drive/sample_train_aug.txt", "rb") as fp:
  sample_aug_list = pickle.load(fp)  

### Defining Train loader, Validation loader, and Test loader

In [ ]:
random.seed(10)


new_images = sample_list + sample_aug_list
new_dens = dens_list + dens_aug_list

train_indices_aug = [] 
valid_indices_aug = random.sample(range(0, len(new_images)), int(0.1*len(new_images)))
for i in range(len(new_images)):
    if i not in valid_indices_aug:
        train_indices_aug.append(i)

new_partition = {'train' : train_indices_aug , 'validation' : valid_indices_aug}

In [ ]:
training_set = Dataset(new_partition['train'], transform, aug=True)

training_loader = torch.utils.data.DataLoader(training_set, batch_size=30,
                                             shuffle=True, num_workers=2)

validation_set = Dataset(new_partition['validation'], transform, aug=True)

validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=30,
                                               shuffle=True, num_workers=2)

### Specifying training options

In [ ]:
torch.manual_seed(10)
# Model 
new_Net = CCNN()
new_Net.to(device)

In [ ]:
optimizer = optim.Adam(new_Net.parameters(), lr = 0.00005)

In [ ]:
max_epochs = 10

In [ ]:
mse = nn.MSELoss()
mae = nn.L1Loss()

### Training main loop

In [ ]:
torch.manual_seed(10)

train_losses = []
validation_losses = []
mse_train_losses = []
mse_valid_losses = []
mae_train_losses = []
mae_valid_losses = []

# Loop over epochs
for epoch in range(max_epochs):
    train_loss = 0
    valid_loss = 0
    batch_loss_train = 0
    batch_loss_valid = 0

    train_mse = 0
    valid_mse = 0
    batch_mse_train = 0
    batch_mse_valid = 0

    train_mae = 0
    valid_mae = 0
    batch_mae_train = 0
    batch_mae_valid = 0

    train_counter = 0
    val_counter = 0 
    
    # Training
    for batch_data, batch_labels in training_loader:

        # Transfer to GPU
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # Model computations
        batch_outputs = new_Net(batch_data)        
        loss = lossfunc(batch_outputs, batch_labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        label = torch.round(torch.sum(batch_labels, dim = [1,2]))
        output = torch.round(torch.sum(batch_outputs.squeeze(), dim = [1,2]))

        train_mse += torch.sqrt(mse(label,output))
        train_mae += torch.sqrt(mae(label,output))

        # print statistics
        

        print('Training: ', 'Epoch No: ', epoch+1, 'Iteration No: ', train_counter+1, '\n',
              'Loss: ', train_loss)
        batch_loss_train += train_loss
        batch_mse_train += train_mse
        batch_mae_train += train_mae  
        train_loss = 0
        train_mse = 0
        train_mae = 0      
        train_counter += 1
        
    # Validation
    with torch.set_grad_enabled(False):
        for batch_data, batch_labels in validation_loader:
            # Transfer to GPU
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
            
            # Model computations
            batch_outputs = new_Net(batch_data)            
            loss = lossfunc(batch_outputs, batch_labels)
            valid_loss += loss.item()
             

            label = torch.round(torch.sum(batch_labels, dim = [1,2]))
            output = torch.round(torch.sum(batch_outputs.squeeze(), dim = [1,2]))

            valid_mse += torch.sqrt(mse(label,output))
            valid_mae += torch.sqrt(mae(label,output))

            print('Validation: ', 'Epoch No: ', epoch+1, 'Iteration No: ', val_counter+1, '\n',
                  'Loss: ', valid_loss)
            batch_loss_valid +=  valid_loss
            batch_mse_valid += valid_mse
            batch_mae_valid += valid_mae
            valid_loss = 0
            valid_mse = 0
            valid_mae = 0   
            val_counter += 1
    
    train_losses.append(batch_loss_train / train_counter)
    validation_losses.append(batch_loss_valid / val_counter)
    mse_train_losses.append(batch_mse_train / train_counter)
    mse_valid_losses.append(batch_mse_valid / val_counter)
    mae_train_losses.append(batch_mae_train / train_counter)
    mae_valid_losses.append(batch_mae_valid / val_counter)

### Saving trained network 

In [ ]:
PATH = '/content/drive/My Drive/CCNN_aug.pth'
torch.save(new_Net.state_dict(), PATH)

### Showing Train & Validation Losses

In [ ]:
plt.plot(train_losses)
plt.plot(validation_losses)
plt.xlabel('Epoch No')
plt.ylabel('Averaged loss')
plt.title("CCNN with augmented data loss function")
plt.legend(labels = ["train","validation"])
plt.show()

In [ ]:
plt.plot(mse_train_losses)
plt.plot(mse_valid_losses)
plt.xlabel('Epoch No')
plt.ylabel('Averaged MSE loss')
plt.title("CCNN with augmented data MSE loss")
plt.legend(labels = ["train","validation"])
plt.show()

In [ ]:
plt.plot(mae_train_losses)
plt.plot(mae_valid_losses)
plt.xlabel('Epoch No')
plt.ylabel('Averaged MAE loss')
plt.title("CCNN with augmented data MAE loss")
plt.legend(labels = ["train","validation"])
plt.show()

# Evaluating the performance on Test data

## Trained network without data augmentation

In [ ]:
PATH = '/content/drive/My Drive/CCNN.pth' 
net_test1 = CCNN()
net_test1.load_state_dict(torch.load(PATH))
net_test1.to(device)

In [ ]:
test_set = Dataset(partition['test'], transform)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=4,
                                             shuffle=True, num_workers=2)

In [ ]:
test_mae = 0
test_mse = 0
test_loss = 0
counter = 0


with torch.set_grad_enabled(False):
        for batch_data, batch_labels in test_loader:
            counter += 1
            # Transfer to GPU
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
            
            # Model computations
            batch_outputs = net_test1(batch_data)            
            loss = lossfunc(batch_outputs, batch_labels)
            test_loss += loss.item()
             

            label = torch.round(torch.sum(batch_labels, dim = [1,2]))
            output = torch.round(torch.sum(batch_outputs.squeeze(), dim = [1,2]))

            if counter == 1:
              sample_batch_data = batch_data
              sample_batch_labels = batch_labels
              sample_batch_outputs = batch_outputs


            test_mse += torch.sqrt(mse(label,output))
            test_mae += torch.sqrt(mae(label,output))

print('Mean loss on test set', test_loss/len(partition['test']))        
print('Mean MSE on test set', test_mse/len(partition['test']))
print('Mean MAE on test set', test_mae/len(partition['test']))        

In [ ]:
img1 = sample_batch_data[0]
label1 = sample_batch_labels[0]
out1 = sample_batch_outputs[0,0]

img2 = sample_batch_data[1]
label2 = sample_batch_labels[1]
out2 = sample_batch_outputs[1,0]


In [ ]:
plt.imshow(img1.cpu().permute(1, 2, 0))
plt.show()

plt.imshow(label1.cpu())
plt.show()

plt.imshow(out1.cpu())
plt.show()

In [ ]:
plt.imshow(img2.cpu().permute(1, 2, 0))
plt.show()

plt.imshow(label2.cpu())
plt.show()

plt.imshow(out2.cpu())
plt.show()

## Trained network with data augmentation

In [ ]:
PATH = '/content/drive/My Drive/CCNN_aug.pth' 
net_test2 = CCNN()
net_test2.load_state_dict(torch.load(PATH))
net_test2.to(device)

In [ ]:
test_mae = 0
test_mse = 0
test_loss = 0
counter = 0


with torch.set_grad_enabled(False):
        for batch_data, batch_labels in test_loader:
            counter += 1
            # Transfer to GPU
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
            
            # Model computations
            batch_outputs = net_test2(batch_data)            
            loss = lossfunc(batch_outputs, batch_labels)
            test_loss += loss.item()
             

            label = torch.round(torch.sum(batch_labels, dim = [1,2]))
            output = torch.round(torch.sum(batch_outputs.squeeze(), dim = [1,2]))

            if counter == 1:
              sample_batch_data = batch_data
              sample_batch_labels = batch_labels
              sample_batch_outputs = batch_outputs


            test_mse += torch.sqrt(mse(label,output))
            test_mae += torch.sqrt(mae(label,output))

print('Mean loss on test set', test_loss/len(partition['test']))        
print('Mean MSE on test set', test_mse/len(partition['test']))
print('Mean MAE on test set', test_mae/len(partition['test']))    

In [ ]:
img1 = sample_batch_data[0]
label1 = sample_batch_labels[0]
out1 = sample_batch_outputs[0,0]

img2 = sample_batch_data[1]
label2 = sample_batch_labels[1]
out2 = sample_batch_outputs[1,0]

In [ ]:
plt.imshow(img1.cpu().permute(1, 2, 0))
plt.show()

plt.imshow(label1.cpu())
plt.show()

plt.imshow(out1.cpu())
plt.show()

In [ ]:
plt.imshow(img2.cpu().permute(1, 2, 0))
plt.show()

plt.imshow(label2.cpu())
plt.show()

plt.imshow(out2.cpu())
plt.show()